### Exploring NLP Models with Skater/LIME

In this example, we'll train a couple types of models, and use Skater, LIME, and ipywidgets to interactively explore model behavior.

### Install Deps

In [1]:
!sudo pip install --upgrade np_utils
!sudo pip install --upgrade theano
!sudo pip install --upgrade tensorflow
!sudo pip install keras==2.0.6
!sudo pip install spacy



    100% |################################| 61kB 2.6MB/s ta 0:00:011
Requirement already up-to-date: numpy>=1.0 in /usr/local/lib/python2.7/dist-packages (from np_utils)
    100% |################################| 829kB 1.4MB/s ta 0:00:011
  Running setup.py bdist_wheel for np-utils ... done
  Stored in directory: /root/.cache/pip/wheels/5d/be/b5/e223535ec3efb733df6afc2518d90d398bbe759e665683b025
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /root/.cache/pip/wheels/c2/50/7c/0d83b4baac4f63ff7a765bd16390d2ab43c93587fac9d6017a
Successfully built np-utils future
    100% |################################| 3.1MB 430kB/s eta 0:00:01
Requirement already up-to-date: numpy>=1.9.1 in /usr/local/lib/python2.7/dist-packages (from theano)
Requirement already up-to-date: scipy>=0.14 in /usr/local/lib/python2.7/dist-packages (from theano)
  Running setup.py bdist_wheel for theano ... done
  Stored in directory: /root/.cache/pip/wheels/d5/5b/93/433299b86e3e9b25f0f600e4e4ebf

    100% |################################| 614kB 2.1MB/s eta 0:00:01
    100% |################################| 51kB 10.2MB/s ta 0:00:01
    100% |################################| 51kB 10.4MB/s ta 0:00:01
    100% |################################| 389kB 3.1MB/s eta 0:00:01
    100% |################################| 61kB 10.8MB/s ta 0:00:01
    100% |################################| 133kB 8.8MB/s eta 0:00:01
    100% |################################| 143kB 8.5MB/s eta 0:00:01
  Running setup.py bdist_wheel for spacy ... done
  Stored in directory: /root/.cache/pip/wheels/8c/a9/96/dba5bf6fdd55d8c04e9ffb1b6244137c36e8b33e03d3e66a9a
  Running setup.py bdist_wheel for preshed ... done
  Stored in directory: /root/.cache/pip/wheels/97/64/22/20fabf1f51039b799e64e46d0381b023cfdbe159c349d7c135
  Running setup.py bdist_wheel for thinc ... done
  Stored in directory: /root/.cache/pip/wheels/ea/1a/6d/0670caa3a983bebfaccf081d9bb092fbd7871d1c4eff8b2c70
  Running setup.py bdist_wheel for pathl

In [2]:
### Restart kernel
from __future__ import absolute_import

!sudo python -m spacy download en



    100% |################################| 52.2MB 110.9MB/s ta 0:00:01
  Running setup.py install for en-core-web-sm ... done

    Linking successful

    /usr/local/lib/python2.7/dist-packages/en_core_web_sm/en_core_web_sm-1.2.0
    --> /usr/local/lib/python2.7/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en').



### Load SpaCy Language Model and Dataset

In [3]:
import spacy
import warnings
from sklearn.datasets import fetch_20newsgroups
import numpy as np
warnings.filterwarnings('ignore')
nlp = spacy.load('en')

from sklearn.model_selection import train_test_split
#gimme data
dataset = fetch_20newsgroups()
docs = dataset.data
y = dataset.target

docs_train, docs_test, y_train, y_test = train_test_split(docs, y, test_size = .3)

### Model 1: Pretrained Word Embeddings

We will use SpaCy's pretrained word embeddings as document representations, and feed these representations into a gradient boosting classifier.

In [4]:
#gimme vectors
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from spacy.tokens.doc import Doc
from sklearn.metrics import classification_report
import six
def doc2vec(x):
    if isinstance(x, (six.binary_type, six.string_types)):
        return nlp(x, parse = False, entity = False, tag = False).vector
    
    elif type(x) in [list, tuple, np.ndarray]:
        return np.array([doc2vec(six.text_type(doc)) for doc in x])
    
    else:
        raise ValueError("Unrecognized Input") 

# build a pipeline of text -> vector (transformer), vector -> predictions (model)
model = GradientBoostingClassifier(n_estimators = 50)
model = LogisticRegression()

transformer = FunctionTransformer(func = doc2vec, validate=False)
pipeline = make_pipeline(transformer, model)
pipeline.fit(docs_train, y_train)       

#Classification Report on Holdout
print(
    classification_report(y_test, 
                          pipeline.predict(docs_test), 
                          target_names=dataset.target_names)
)

                          precision    recall  f1-score   support

             alt.atheism       0.57      0.44      0.49       160
           comp.graphics       0.45      0.48      0.46       167
 comp.os.ms-windows.misc       0.66      0.51      0.58       204
comp.sys.ibm.pc.hardware       0.58      0.54      0.56       180
   comp.sys.mac.hardware       0.59      0.48      0.53       170
          comp.windows.x       0.58      0.72      0.65       182
            misc.forsale       0.66      0.75      0.70       175
               rec.autos       0.78      0.60      0.68       175
         rec.motorcycles       0.58      0.74      0.65       180
      rec.sport.baseball       0.72      0.67      0.69       183
        rec.sport.hockey       0.73      0.83      0.78       164
               sci.crypt       0.68      0.78      0.72       186
         sci.electronics       0.65      0.56      0.60       162
                 sci.med       0.77      0.91      0.83       184
         

### Model 2: CNN 
In this model, we convert text to a list of padded lists of word IDs, to be used in an embedding lookup table. The embeddings will be trained as part of a CNN implemented with Keras.

In [8]:

from sklearn.datasets import fetch_20newsgroups
from spacy.tokens import Doc
import spacy
from spacy.matcher import Matcher
from spacy.attrs import ORTH, IS_PUNCT
from collections import Counter
from functools import partial

class TextProcesser(object):
    def __init__(self, corpus, nlp=None, max_len=200, max_vocab_size=20000):
        """
        corpus: list of strings
            Documents used to initialize vocabulary.
            
        nlp: Spacy language model
            If none then will build one in __init__
            
        max_len: int
            Maximum length of a document sequence. Balance information with scale of data.
            
        max_vocab_size: int
        
        """
        self.max_vocab_size = max_vocab_size
        self.max_len = max_len
        self.nlp = nlp or spacy.load('en')
        self.PADDING_VAL = 1
        self.MISSING_VAL = 2
        self.START_VAL = 3
        self.END_VAL = 4
        self.vocab = {}
        self.vocab_counts = Counter()#Counter(['PADDING_VAL','MISSING_VAL','START_VAL','END_VAL'])
        self.build_vocab(corpus)

        
    def pad(self, obj):
        n_pads = max(self.max_len - len(obj) - 2, 0)
        we_can_take = self.max_len - 2
        result = [self.START_VAL] + obj[:we_can_take] + [self.END_VAL] + [self.PADDING_VAL] * n_pads
        return result
        
    def get_current_vocab_size(self):
        return len(self.vocab)
        
    def update(self, words):
        for word in words:
            self.vocab_counts.update([word])

    def build_vocab(self, corpus):
        self.vocab = {}
        self.vocab_counts = Counter()
        
        for doc in nlp.tokenizer.pipe(map(six.text_type, corpus)):
            self.update(map(self._process_token, doc))
            
        for i, (word, count) in enumerate(self.vocab_counts.most_common(self.max_vocab_size)):
            self.vocab[word] = i
        
    def _process_token(self, token):
        if token.is_space:
            return "SPACE"
        elif token.is_punct:
            return "PUNCT"       
        elif token.like_url:
            return "URL"
        elif token.like_email:
            return "EMAIL"
        elif token.like_num:
            return "NUM"
        else:
            return token.lower_

    def process_token(self, token):
        return self.vocab.get(self._process_token(token), self.MISSING_VAL)

    def process(self, texts):
        docs = []
        for doc in self.nlp.tokenizer.pipe(list(texts)):
            docs.append(self.pad(list(map(self.process_token, doc))))
        return np.array(docs)
            
    def __call__(self, texts):
        return self.process(texts)
    

In [9]:
#convolutional model: https://arxiv.org/abs/1408.5882
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout,  Input, Dense, Activation, Flatten
from keras.models import Sequential, Model, Sequential
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import TimeDistributed
from keras.layers.merge import Concatenate
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import label_binarize


def model_factory(seq_len, 
                  vocab_size, 
                  embedding_size, 
                  n_classes, 
                  model_type='sequential',
                  loss='categorical_crossentropy', 
                  metrics=['acc'], 
                  optimizer='rmsprop'):
    
    def create_sequential_model():
        model = Sequential()
        model.add(Embedding(vocab_size, embedding_size, input_length=seq_len))
        model.add(Conv1D(64, 3, strides=1, padding='valid'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(GlobalMaxPooling1D())
        model.add(Activation('relu'))
        model.add(Dense(n_classes,  activation='softmax'))
        return model
        
    def create_non_sequential_model():
        _input = Input(shape=(seq_len,), dtype='int32')
        _embedding = Embedding(vocab_size, embedding_size, input_length=seq_len)(_input)

        # each filter is (3 x 300 ) array of weights
        # window (kernel_size) is 3
        # so number of weights is (3 * 300 * 64)
        # each filter outputs a (200 / strides) x 1 transformation
        # padding is how we handle boundaries. include + pad, ignore, etc
        _conv_1 = Conv1D(64, 3, strides=1, padding='valid')(_embedding)

        # Cuts the size of the output in half, maxing over every 2 inputs
        _pool_1 = MaxPooling1D(pool_size=2)(_conv_1)
        _conv_2 = Conv1D(64, 3, padding='valid')(_pool_1)
        _pool_2 = GlobalMaxPooling1D()(_conv_2) 
        _activation = Activation('relu')(_pool_2)
        output = Dense(n_classes,  activation='softmax')(_activation)
        model = Model(inputs=_input, outputs=output)
        return model
        

    def create_model():
        if model_type=='sequential':
            model = create_sequential_model()
        elif model_type == 'non-sequential':
            model = create_non_sequential_model()        
        else:
            raise ValueError("Unrecognized model type {}".format(model_type))

        model.compile(loss=loss,
                     optimizer=optimizer,
                     metrics=metrics)
        return model

    return create_model
    
seq_len = 350
vocab_size = 25000
embedding_size = 300
epochs = 8
batch_size = 100
n_classes = len(np.unique(y))

model_build = model_factory(seq_len, vocab_size, embedding_size, n_classes)
model2 = KerasClassifier(build_fn=model_build, epochs=epochs, batch_size=batch_size, verbose=1)
processor = FunctionTransformer(TextProcesser(docs_train, nlp=nlp, max_len=seq_len), validate=False)
pipeline2 = make_pipeline(processor, model2)

# need to one hot encode y labels
y2_train = label_binarize(y_train, classes=range(len(np.unique(dataset.target_names))))
pipeline2.fit(docs_train, y2_train)

# make model silent after training
params = model2.get_params()
params = {key: value for key, value in params.items() if key != 'build_fn'}
params['verbose'] = 0
model2.set_params(**params)

# Model Performance on Holdout
print(
    classification_report(y_test, 
                          pipeline2.predict(docs_test), 
                          target_names=dataset.target_names)
)

Using TensorFlow backend.


Epoch 1/8
7919/7919 [==============================] - 40s - loss: 2.7191 - acc: 0.3396    
Epoch 2/8
7919/7919 [==============================] - 38s - loss: 1.6840 - acc: 0.6862    
Epoch 3/8
7919/7919 [==============================] - 40s - loss: 0.9258 - acc: 0.8067    
Epoch 4/8
7919/7919 [==============================] - 40s - loss: 0.5595 - acc: 0.8775    
Epoch 5/8
7919/7919 [==============================] - 39s - loss: 0.3478 - acc: 0.9261    
Epoch 6/8
7919/7919 [==============================] - 39s - loss: 0.2140 - acc: 0.9601    
Epoch 7/8
7919/7919 [==============================] - 40s - loss: 0.1251 - acc: 0.9792    
Epoch 8/8
7919/7919 [==============================] - 41s - loss: 0.0699 - acc: 0.9904    
                          precision    recall  f1-score   support

             alt.atheism       0.90      0.87      0.88       151
           comp.graphics       0.84      0.80      0.82       196
 comp.os.ms-windows.misc       0.84      0.81      0.83       161

### Model explanations
Here, we'll wrap each pipeline into a Skater model object. We'll use this model object to generate LIME explanations in HTML to help better understand how each model makes predictions. We'll wrap this functionality into an ipywidget to allow the user the (a) modify the text and (b) toggle between models.

In [5]:
## You may need to enable ipywidgets
!sudo pip install ipywidgets
!jupyter nbextension enable --py --user widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [7]:
!sudo pip install skater

    100% |################################| 256kB 3.0MB/s ta 0:00:01
    100% |################################| 71kB 4.4MB/s ta 0:00:011
    100% |################################| 81kB 4.7MB/s ta 0:00:011
    100% |################################| 51kB 8.7MB/s eta 0:00:01
    100% |################################| 1.5MB 872kB/s eta 0:00:01
  Running setup.py bdist_wheel for skater ... done
  Stored in directory: /root/.cache/pip/wheels/2b/80/c4/27748f133a062c246ef28f8916ed8c7bd7f4dc040f3d894be0
  Running setup.py bdist_wheel for ds-lime ... done
  Stored in directory: /root/.cache/pip/wheels/a8/fe/5a/7a7bfca3a84f80ad75d6885ee6e78a887e0b0b480bb2ff0cc6
  Running setup.py bdist_wheel for pathos ... done
  Stored in directory: /root/.cache/pip/wheels/ac/dd/95/a910070ff4be5079956b2764f61bb8b05e37c7e7563b5e7848
  Running setup.py bdist_wheel for ppft ... done
  Stored in directory: /root/.cache/pip/wheels/4d/28/cc/c5bca1e1d3923a5f4dd50219a9e41391bbeb832835e7905344
  Running setup.py bdis

In [8]:
#Create the explorer app.
from warnings import filterwarnings
filterwarnings('ignore')
from ipywidgets import Button, Textarea, Layout, Box, Label, Text, Output, RadioButtons, HBox
from IPython.display import display, HTML, clear_output
from skater.core.local_interpretation.lime.lime_text import LimeTextExplainer

class TextExplainer(object):
    def __init__(self, models, init_pattern=""):
        """
        Display box for LIME results.
        
        models: dictionary of skater of models.
            Keys correspond to user-defined model names, used for radio buttons.
            Values are skater models used to generate predictions.
    
        """
        self.status = "Ready"
        self.explainer = LimeTextExplainer(class_names=dataset.target_names)
        self.models = models
        self.model_names = list(self.models.keys())
        self.text_field = Textarea(init_pattern, layout=Layout(height='200px', width='500px'))
        self.text_box = Box([Label(value='Text Box'), self.text_field])
        
        self.status_field = Label(self.status, layout=Layout(height='50px', width='100px'))        
        self.status_box = Box([Label(value='Status'), self.status_field])

        self.match_button = Button(description='Explain', )
        self.match_button.on_click(self.match_pattern)
        
        self.model_selectors = RadioButtons(
            options = self.model_names,
            description = "Use Model"
        )
        
        self.inputs_box = HBox([self.text_box, self.model_selectors])        
        
        self.explanation_area = Output()
        display(self.inputs_box)       
        display(self.match_button)
        display(self.status_box)
        display(self.explanation_area)
        
    @property
    def model(self):
        return self.models[self.model_selectors.value]
            
    @property
    def text(self):
        return self.text_field.value    
    
    def match_pattern(self, b):
        self.status_field.value = 'loading'
        with self.explanation_area:
            clear_output()
            display(HTML(self.get_explanation_as_html(self.text)))
        self.status_field.value = 'ready'

    def get_explanation_as_html(self, text):
        
        # generate most likely class to confine LIME results

        explanation = self.explainer.explain_instance(text, 
                                                      self.model, 
                                                      top_labels=1)

        return explanation.as_html()


In [10]:
models = {"CNN": pipeline2.predict_proba, ' GBC-Pretrain': pipeline.predict_proba}
r = TextExplainer(models, docs_test[3])

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget